In [0]:
%pip install -r ../requirements.txt
dbutils.library.restartPython()

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import sys
import os
sys.path.append(os.path.abspath('../chaosllama'))
os.chdir(os.path.abspath('../'))
print(os.getcwd())

In [0]:
from chaosllama.chaos import ChaosLlama, ChaosLlamaServicesConfig
from chaosllama.entities.models import AgentConfig
from chaosllama.services import genie, mosaic, unity_catalog, judges
from chaosllama.services.evaluation_dataset import EvalSetManager
from chaosllama.profiles.config import config
from chaosllama.services.tracking import MLFlowExperimentManager
from chaosllama.scorers.scorers import eval_sql_clauses_distro, eval_query_results
import chaosllama.prompts.registry as prompt_registry
import pyfiglet
import rich
from rich.console import Console

In [0]:
# 🪵Logging Configuration
# TODO: Add logging configuration

banner = pyfiglet.figlet_format("Chaos Llama 🦙")
print(banner)

# 🧑‍🍳 Prepare Evaluation Data Set
evmngr = EvalSetManager(
    table_name=f"{config.CATALOG}.{config.SCHEMA}.{config.EVAL_TABLE_NAME}",
    limit=config.runtime.LIMIT,
    consistency_factor=config.runtime.CONSISTENCY_FACTOR
)

evmngr.prepare_evals(mode="existing")
# evmngr.write_evalset()

# 🧑‍🔬Set MLFLow Experiment
exp_mngr = MLFlowExperimentManager(experiment_path=config.mlflow.MLFLOW_EXPERIMENT_PATH).get_or_create_mlflow_experiment(config.mlflow.MLFLOW_RUNTIME_EXPERIMENT)

# =========Configure Services=============
#🧑‍⚖️Judges - Manages custom judges 
jmngr = judges.JudgeService(scorers=[eval_sql_clauses_distro,eval_query_results]) 
# 🧪Mosaic Evaluations - Manages MLFlow Experiments
mlfmngr = mosaic.MosaicEvalService(eval_manager=evmngr,judge_manager=jmngr, experiment_id=exp_mngr.experiment_id) 
# 🧞‍♂️Genie Service - Manages Interactions with Genie
gmngr = genie.GenieService(space_id=config.genie.RUNTIME_GENIE_SPACE_ID) 
# 🤝 Unity Catalog Manager - Manages Interactions with UC 
ucmngr = unity_catalog.UCService(catalog=config.CATALOG, schema=config.SCHEMA) 
# =========================================

agent_config = AgentConfig(
        system_prompt=prompt_registry.INSTROSPECT_PROMPT_V3,
        endpoint=config.runtime.INTROSPECT_AGENT_LLM_ENDPOINT,
        llm_parameters={"temperature": 0.0, "max_tokens": config.runtime.MAX_TOKENS},
)

# ⚙️🦙Configure Chaos Llama
chaos_config = ChaosLlamaServicesConfig(
    mlflow_manager=mlfmngr,
    genie_manager=gmngr,
    uc_manager=ucmngr,
    agent_config=agent_config
)

chaos_llama = ChaosLlama(config=chaos_config)

data_intelligence, mlflow_parent_run = chaos_llama.run(
    epochs=config.runtime.EPOCHS,
    is_test=config.runtime.DEBUG,
    is_cached=config.runtime.IS_CACHED,
    run_baseline=config.runtime.RUN_BASELINE,
    run_null_hypothesis=config.runtime.RUN_NULL_HYPOTHESIS,
)